In [1]:
from getDistance import getDistance
from getElevation import getElevation
from getCentroid import getCentroid
from getAzimuth import getAzimuth

In [2]:
import numpy as np
import dicom
import sys
sys.path.append('..')
import os
from ovh import getOVHDistances, getHistogram, getNormalizedHistogram
from utils import getVolume, getContours
from math import sqrt
import time
start_time = time.time() # For runtime test
import matplotlib.pyplot as plt

## Inputs to Function

In [3]:
BASE_DIR = '/home/radiation/RadiationTherapyDecisionSupport/data/'
PatientID = 'UCLA_PR_5'

ctFilenames = [fl for fl in os.listdir(BASE_DIR + PatientID) if 'CT.' in fl]
numImages = len(ctFilenames)

sampleCTImage = dicom.read_file(BASE_DIR + PatientID + '/' + ctFilenames[0])
width = sampleCTImage.Columns
height = sampleCTImage.Rows
row_spacing = float(sampleCTImage.PixelSpacing[0])
column_spacing = float(sampleCTImage.PixelSpacing[1])
slice_spacing = float(sampleCTImage.SliceThickness)

block_shape = (width, height, numImages)
slice_position_z = np.zeros((numImages)).astype(np.float32) 

for i, fl in enumerate(ctFilenames):
    slice_position_z[i] = dicom.read_file(BASE_DIR + PatientID + '/' + fl).ImagePositionPatient[2]

In [4]:
structureset = dicom.read_file(BASE_DIR + PatientID + '/structureset.dcm')
PTV_ROI_NUM = -1
OAR_ROI_NUM = -1

for n in range(0, len(structureset.StructureSetROISequence)):
    if structureset.StructureSetROISequence[n].ROIName == 'PTV':
        PTV_ROI_NUM = structureset.StructureSetROISequence[n].ROINumber
    elif structureset.StructureSetROISequence[n].ROIName == 'Bladder':
        OAR_ROI_NUM = structureset.StructureSetROISequence[n].ROINumber

In [5]:
roiNumPlanes = len(structureset.ROIContourSequence[PTV_ROI_NUM].ContourSequence) 

contour_data = {} 
image_orientation = {} 
image_position = {} 
pixel_spacing = {} 

for index in range(0, roiNumPlanes):
    
    imageSOP = structureset.ROIContourSequence[PTV_ROI_NUM].ContourSequence[index].ContourImageSequence[0].ReferencedSOPInstanceUID
    
    planeContourData = np.array(structureset.ROIContourSequence[PTV_ROI_NUM].ContourSequence[index].ContourData)
    planeContourData = planeContourData.reshape(planeContourData.shape[0] // 3 , 3)
    
    contour_data[imageSOP] = planeContourData
    imagei = dicom.read_file(BASE_DIR + PatientID + '/CT.' + imageSOP + '.dcm')
    
    image_orientation[imageSOP] = imagei.ImageOrientationPatient
    image_position[imageSOP] = imagei.ImagePositionPatient
    pixel_spacing[imageSOP] = imagei.PixelSpacing
ptv_contour_block, ptv_roi_block = getContours(block_shape, slice_position_z, contour_data, image_orientation, image_position, pixel_spacing)

In [6]:
roiNumPlanes = len(structureset.ROIContourSequence[OAR_ROI_NUM].ContourSequence) 

contour_data = {}
image_orientation = {}
image_position = {} 
pixel_spacing = {}

for index in range(0, roiNumPlanes):
    
    imageSOP = structureset.ROIContourSequence[OAR_ROI_NUM].ContourSequence[index].ContourImageSequence[0].ReferencedSOPInstanceUID
    planeContourData = np.array(structureset.ROIContourSequence[OAR_ROI_NUM].ContourSequence[index].ContourData)
    planeContourData = planeContourData.reshape(planeContourData.shape[0] // 3 , 3)
    
    contour_data[imageSOP] = planeContourData
    imagei = dicom.read_file(BASE_DIR + PatientID + '/CT.' + imageSOP + '.dcm')
    
    image_orientation[imageSOP] = imagei.ImageOrientationPatient
    image_position[imageSOP] = imagei.ImagePositionPatient
    pixel_spacing[imageSOP] = imagei.PixelSpacing
_, oar_roi_block = getContours(block_shape, slice_position_z, contour_data, image_orientation, image_position, pixel_spacing)

In [7]:
centroid = getCentroid(oar_roi_block)

In [8]:
elevation = np.zeros(np.count_nonzero(ptv_roi_block))
distance = np.zeros(np.count_nonzero(ptv_roi_block))
azimuth = np.zeros(np.count_nonzero(ptv_roi_block))
count = 0

In [9]:
for i in range(0, ptv_roi_block.shape[0]):
    for j in range(0, ptv_roi_block.shape[1]):
        for k in range(0, ptv_roi_block.shape[2]):
            if ptv_roi_block[i,j,k] == 1:
                elevation[count] = getElevation(centroid, [i,j,k])
                distance[count] = getDistance(centroid, [i,j,k])
                azimuth[count] = getAzimuth(centroid, [i,j,k])
                count = count+1;

In [10]:
n_bins = 10;
elevation_max = np.max(elevation)
elevation_min = np.min(elevation)
distance_max = np.max(distance)
distance_min = np.min(distance)
azimuth_max = np.max(azimuth)
azimuth_min = np.min(azimuth)
elevation_delta = (elevation_max-elevation_min)/n_bins
distance_delta = (distance_max-distance_min)/n_bins
azimuth_delta = (azimuth_max-azimuth_min)/n_bins

In [19]:
elevation_bins = np.concatenate((np.arange(elevation_min,elevation_max,elevation_delta), 
                                 np.expand_dims(np.array(elevation_max), axis=0)))
distance_bins = np.concatenate((np.arange(distance_min,distance_max,distance_delta), 
                                np.expand_dims(np.array(distance_max), axis=0)))
azimuth_bins = np.concatenate((np.arange(azimuth_min,azimuth_max,azimuth_delta), 
                               np.expand_dims(np.array(azimuth_max), axis=0)))
amounts = np.zeros((elevation_bins.shape[0], distance_bins.shape[0], azimuth_bins.shape[0]))

In [22]:
for i in range(0, elevation_bins.shape[0] - 1):
    for j in range(0, distance_bins.shape[0] - 1):
        for k in range(0, azimuth_bins.shape[0] - 1):
            amounts[i,j,k] = np.count_nonzero((elevation >= elevation_bins[i]) & (elevation< elevation_bins[i+1]) & (distance >= distance_bins[j]) & (distance < distance_bins[j+1]) & (azimuth >= azimuth_bins[k]) & (azimuth < azimuth_bins[k+1]))

In [23]:
amounts = amounts/getVolume(ptv_roi_block)